# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = '../WeatherPy/city_data.csv'
city_data_df = pd.read_csv(csv_path)
clean_df = city_data_df.dropna()
clean_df = clean_df.drop(columns='Unnamed: 0')
clean_df = clean_df.drop_duplicates()
clean_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.88,147.33,44.60,70,75,17.22,AU,1596611250
1,Rikitea,-23.12,-134.97,72.90,90,96,12.01,PF,1596611252
2,Ilulissat,69.22,-51.10,42.80,87,98,8.05,GL,1596611253
3,Khovd,48.01,91.64,71.13,43,61,7.70,MN,1596611066
4,Hilo,19.73,-155.09,75.20,73,90,5.82,US,1596611256


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [20]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_df[["Lat", "Lng"]].astype(float)
weights = clean_df['Humidity'].astype(float)

In [21]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_cities = clean_df[((clean_df['Max Temp'] < 75)&(clean_df['Max Temp'] >68)) & (clean_df['Wind Speed'] < 10 ) & (clean_df['Cloudiness'] == 0) & (clean_df['Humidity'] < 50)]
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
133,Beloha,-25.17,45.05,74.44,48,0,7.23,MG,1596611510
181,Nortelândia,-14.45,-56.80,69.01,41,0,4.56,BR,1596611599
265,Sovetskiy,61.36,63.58,73.92,39,0,3.15,RU,1596611756
355,Mwinilunga,-11.74,24.43,72.12,29,0,3.76,ZM,1596611928
415,Benjamín Aceval,-24.97,-57.57,69.80,49,0,9.17,PY,1596612014
444,Igrim,63.19,64.42,73.40,40,0,5.35,RU,1596612032
525,Porto Murtinho,-21.70,-57.88,68.77,40,0,5.01,BR,1596612205


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_cities[['City','Country','Lat','Lng']]
hotel_df['Hotel Name'] =''
hotel_df

C:\Users\debby\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
133,Beloha,MG,-25.17,45.05,
181,Nortelândia,BR,-14.45,-56.80,
265,Sovetskiy,RU,61.36,63.58,
355,Mwinilunga,ZM,-11.74,24.43,
415,Benjamín Aceval,PY,-24.97,-57.57,
444,Igrim,RU,63.19,64.42,
525,Porto Murtinho,BR,-21.70,-57.88,


In [8]:
partial_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
parameters = {
    'key': g_key,
    'type': 'Lodging',
    'radius': 5000,
    'keyword' : 'Hotel'
}

In [9]:
for index, row in hotel_df.iterrows():
    lat_r = row[2]
    lng_r = row[3]
    location = f'{lat_r},{lng_r}'
    
    parameters['location'] = location
    
    response = requests.get(partial_url, params = parameters).json()
    hotel_results = response['results']
    
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_results[0]['name']
        
    except:
        pass
#     print(json.dumps(response,indent=4))
#     print(hotel_results)
hotel_df 

C:\Users\debby\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Hotel Name
133,Beloha,MG,-25.17,45.05,
181,Nortelândia,BR,-14.45,-56.80,Hotel Karysmã
265,Sovetskiy,RU,61.36,63.58,Hotel Alexandria
355,Mwinilunga,ZM,-11.74,24.43,Adayi Lodge
415,Benjamín Aceval,PY,-24.97,-57.57,Cabaña La Celia
444,Igrim,RU,63.19,64.42,Obshchezhitiye Uchebno-Proizvodstvennogo Tsentra
525,Porto Murtinho,BR,-21.70,-57.88,Hotel Pantanal


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))